# Marketing A/B Test - Bayesian Statistical Analysis

**Author**: Analytics Team  
**Date**: November 2025  
**Version**: 1.0

---

## Executive Summary

This notebook performs Bayesian statistical analysis using the Beta-Binomial model to estimate conversion rates and calculate probabilities of superiority. Bayesian methods provide intuitive probability statements that are directly interpretable for business decision-making.

## Bayesian vs Frequentist Approach

### Key Advantages of Bayesian Analysis

- **Direct Probability Statements**: "There is a 95% probability that the ad group is superior"
- **Prior Knowledge Integration**: Can incorporate historical data or expert knowledge
- **Sequential Updates**: Beliefs update as new data arrives
- **Posterior Distributions**: Full uncertainty quantification, not just point estimates
- **Intuitive Interpretation**: Credible intervals have natural probability interpretation

### Beta-Binomial Model

We model conversion rates using a Beta-Binomial conjugate prior, which is mathematically elegant and computationally efficient for binary outcomes. The Beta distribution is the natural conjugate prior for the Binomial likelihood.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import beta
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


## 1. Data Preparation

Load data and extract conversion counts for each group.


In [ ]:
# Load data
df = pd.read_csv('marketing_AB.csv')

# Separate groups
ad_group = df[df['test group'] == 'ad']
psa_group = df[df['test group'] == 'psa']

# Extract conversion data
ad_conversions = ad_group['converted'].sum()
ad_non_conversions = len(ad_group) - ad_conversions
n_ad = len(ad_group)

psa_conversions = psa_group['converted'].sum()
psa_non_conversions = len(psa_group) - psa_conversions
n_psa = len(psa_group)

# Observed conversion rates
cr_ad = ad_conversions / n_ad
cr_psa = psa_conversions / n_psa

print(f"Ad Group:  {n_ad:,} users, {ad_conversions:,} conversions, Rate: {cr_ad:.6f}")
print(f"PSA Group: {n_psa:,} users, {psa_conversions:,} conversions, Rate: {cr_psa:.6f}")


## 2. Beta-Binomial Model

### Prior Distribution

We use a non-informative uniform prior: $\text{Beta}(1, 1)$, which is equivalent to a uniform distribution on $[0,1]$.

The Beta distribution has probability density function:

$$f(p | \alpha, \beta) = \frac{p^{\alpha-1}(1-p)^{\beta-1}}{B(\alpha, \beta)}$$

where $B(\alpha, \beta)$ is the Beta function.

### Posterior Distribution

For a Beta prior and Binomial likelihood, the posterior is also Beta (conjugate prior):

$$\text{Posterior} = \text{Beta}(\alpha + \text{successes}, \beta + \text{failures})$$

### Posterior Mean

The posterior mean (expected conversion rate) is:

$$E[p | \text{data}] = \frac{\alpha + \text{successes}}{\alpha + \beta + \text{total trials}}$$


In [ ]:
# Prior parameters (non-informative uniform prior: Beta(1, 1))
alpha_prior_ad = 1
beta_prior_ad = 1
alpha_prior_psa = 1
beta_prior_psa = 1

# Posterior parameters (Beta(alpha + successes, beta + failures))
alpha_post_ad = alpha_prior_ad + ad_conversions
beta_post_ad = beta_prior_ad + ad_non_conversions

alpha_post_psa = alpha_prior_psa + psa_conversions
beta_post_psa = beta_prior_psa + psa_non_conversions

print("BETA-BINOMIAL MODEL")
print("="*60)
print(f"Prior Distribution:")
print(f"  Ad Group:  Beta({alpha_prior_ad}, {beta_prior_ad})")
print(f"  PSA Group: Beta({alpha_prior_psa}, {beta_prior_psa})")

print(f"\nPosterior Distribution:")
print(f"  Ad Group:  Beta({alpha_post_ad:.1f}, {beta_post_ad:.1f})")
print(f"  PSA Group: Beta({alpha_post_psa:.1f}, {beta_post_psa:.1f})")

# Posterior means
post_mean_ad = alpha_post_ad / (alpha_post_ad + beta_post_ad)
post_mean_psa = alpha_post_psa / (alpha_post_psa + beta_post_psa)

print(f"\nPosterior Mean Conversion Rates:")
print(f"  Ad Group:  {post_mean_ad:.6f} ({post_mean_ad*100:.4f}%)")
print(f"  PSA Group: {post_mean_psa:.6f} ({post_mean_psa*100:.4f}%)")
print(f"  Expected Lift: {(post_mean_ad - post_mean_psa) / post_mean_psa * 100:.4f}%")


## 3. Credible Intervals

Credible intervals are the Bayesian equivalent of confidence intervals, but with a more intuitive interpretation.

### Credible Interval

A $(1-\alpha)$ credible interval contains $(1-\alpha)$ of the posterior probability:

$$P(p \in [L, U] | \text{data}) = 1 - \alpha$$

We can find $L$ and $U$ such that:
- $P(p < L | \text{data}) = \alpha/2$
- $P(p > U | \text{data}) = \alpha/2$

For a Beta distribution, we use the inverse CDF (percentile function).


In [ ]:
# Visualization: Posterior Distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Posterior distributions
x = np.linspace(0.10, 0.12, 1000)
post_ad = beta.pdf(x, alpha_post_ad, beta_post_ad)
post_psa = beta.pdf(x, alpha_post_psa, beta_post_psa)

axes[0].plot(x, post_ad, label='Ad Group Posterior', linewidth=2.5, color='#3B82F6')
axes[0].plot(x, post_psa, label='PSA Group Posterior', linewidth=2.5, color='#6B7280')
axes[0].axvline(post_mean_ad, color='#3B82F6', linestyle='--', linewidth=2, alpha=0.7, label=f'Ad Mean: {post_mean_ad:.4f}')
axes[0].axvline(post_mean_psa, color='#6B7280', linestyle='--', linewidth=2, alpha=0.7, label=f'PSA Mean: {post_mean_psa:.4f}')
axes[0].fill_between(x, post_ad, alpha=0.3, color='#3B82F6')
axes[0].fill_between(x, post_psa, alpha=0.3, color='#6B7280')
axes[0].set_xlabel('Conversion Rate', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Probability Density', fontsize=12, fontweight='bold')
axes[0].set_title('Posterior Distributions of Conversion Rates', fontsize=14, fontweight='bold', pad=20)
axes[0].legend(fontsize=11)
axes[0].grid(alpha=0.3, linestyle='--')

# Difference distribution
diff_x = np.linspace(posterior_diff_samples.min(), posterior_diff_samples.max(), 100)
axes[1].hist(posterior_diff_samples, bins=50, density=True, alpha=0.7, color='green', edgecolor='black', linewidth=1.5)
axes[1].axvline(0, color='red', linestyle='--', linewidth=2, label='Zero (No Difference)')
axes[1].axvline(diff_ci_lower, color='orange', linestyle='--', linewidth=2, label=f'95% CI Lower: {diff_ci_lower:.6f}')
axes[1].axvline(diff_ci_upper, color='orange', linestyle='--', linewidth=2, label=f'95% CI Upper: {diff_ci_upper:.6f}')
axes[1].set_xlabel('Difference (Ad - PSA)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Probability Density', fontsize=12, fontweight='bold')
axes[1].set_title('Posterior Distribution of Difference', fontsize=14, fontweight='bold', pad=20)
axes[1].legend(fontsize=10)
axes[1].grid(alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()


In [ ]:
# Generate samples from posterior distributions
n_samples = 100000
posterior_ad_samples = np.random.beta(alpha_post_ad, beta_post_ad, n_samples)
posterior_psa_samples = np.random.beta(alpha_post_psa, beta_post_psa, n_samples)

# Difference distribution
posterior_diff_samples = posterior_ad_samples - posterior_psa_samples

# 95% Credible intervals
ci_level = 0.95
alpha_ci = 1 - ci_level

# For Ad group
ad_ci_lower = np.percentile(posterior_ad_samples, 100 * alpha_ci / 2)
ad_ci_upper = np.percentile(posterior_ad_samples, 100 * (1 - alpha_ci / 2))

# For PSA group
psa_ci_lower = np.percentile(posterior_psa_samples, 100 * alpha_ci / 2)
psa_ci_upper = np.percentile(posterior_psa_samples, 100 * (1 - alpha_ci / 2))

# For difference
diff_ci_lower = np.percentile(posterior_diff_samples, 100 * alpha_ci / 2)
diff_ci_upper = np.percentile(posterior_diff_samples, 100 * (1 - alpha_ci / 2))

print("CREDIBLE INTERVALS (95%)")
print("="*60)
print(f"Ad Group Conversion Rate:")
print(f"  [{ad_ci_lower:.6f}, {ad_ci_upper:.6f}]")
print(f"  [{(ad_ci_lower*100):.4f}%, {(ad_ci_upper*100):.4f}%]")

print(f"\nPSA Group Conversion Rate:")
print(f"  [{psa_ci_lower:.6f}, {psa_ci_upper:.6f}]")
print(f"  [{(psa_ci_lower*100):.4f}%, {(psa_ci_upper*100):.4f}%]")

print(f"\nDifference (Ad - PSA):")
print(f"  [{diff_ci_lower:.6f}, {diff_ci_upper:.6f}]")
print(f"  [{(diff_ci_lower*100):.4f}%, {(diff_ci_upper*100):.4f}%]")

if diff_ci_lower > 0:
    print(f"\n✅ Credible interval excludes zero - Ad group is superior")
elif diff_ci_upper < 0:
    print(f"\n✅ Credible interval excludes zero - PSA group is superior")
else:
    print(f"\n⚠️  Credible interval includes zero - No clear superiority")


## 4. Probability of Superiority

One of the key advantages of Bayesian analysis is the ability to make direct probability statements.

### Probability of Superiority

We calculate:

$$P(p_{\text{ad}} > p_{\text{psa}} | \text{data})$$

This is the probability that the ad group has a higher conversion rate than the psa group, given the observed data.

We estimate this by sampling from the posterior distributions and calculating the proportion of samples where $p_{\text{ad}} > p_{\text{psa}}$.


In [ ]:
# Probability that Ad > PSA
prob_ad_better = np.mean(posterior_ad_samples > posterior_psa_samples)
prob_psa_better = np.mean(posterior_psa_samples > posterior_ad_samples)
prob_equal = 1 - prob_ad_better - prob_psa_better

print("PROBABILITY OF SUPERIORITY")
print("="*60)
print(f"P(Ad > PSA): {prob_ad_better:.6f} ({prob_ad_better*100:.4f}%)")
print(f"P(PSA > Ad): {prob_psa_better:.6f} ({prob_psa_better*100:.4f}%)")
print(f"P(Equal):    {prob_equal:.6f} ({prob_equal*100:.4f}%)")

# Probability of meaningful lift (e.g., > 1% relative lift)
min_lift = 0.01
prob_meaningful_lift = np.mean((posterior_ad_samples - posterior_psa_samples) / posterior_psa_samples > min_lift)
print(f"\nProbability of >{min_lift*100:.0f}% relative lift:")
print(f"  P(Lift > {min_lift*100:.0f}%): {prob_meaningful_lift:.6f} ({prob_meaningful_lift*100:.4f}%)")

# Interpretation
if prob_ad_better > 0.95:
    interpretation = "Very Strong Evidence"
elif prob_ad_better > 0.90:
    interpretation = "Strong Evidence"
elif prob_ad_better > 0.75:
    interpretation = "Moderate Evidence"
elif prob_ad_better > 0.50:
    interpretation = "Weak Evidence"
else:
    interpretation = "No Evidence"

print(f"\n{interpretation} that Ad group is superior")


## 5. Expected Value Calculations

We can use the posterior distribution to estimate expected business impact.

### Expected Incremental Conversions

$$\text{Expected Incremental Conversions} = (E[p_{\text{ad}}] - E[p_{\text{psa}}]) \times n_{\text{ad}}$$

### Expected Incremental Revenue

$$\text{Expected Revenue} = \text{Expected Incremental Conversions} \times \text{Value per Conversion}$$

We can also calculate the full distribution of expected revenue by sampling from the posterior.


## Summary & Conclusions

### Bayesian Analysis Results
- **Probability Ad > PSA**: {prob_ad_better:.2%}
- **Expected Lift**: {(post_mean_ad - post_mean_psa) / post_mean_psa * 100:.4f}%
- **Expected Incremental Revenue**: ${expected_incremental_revenue:,.2f}
- **95% Credible Interval**: [{diff_ci_lower:.6f}, {diff_ci_upper:.6f}]

### Key Findings
- {interpretation} that Ad group is superior
- The posterior distribution provides full uncertainty quantification
- Business impact can be directly estimated from the posterior

### Recommendations
Based on the Bayesian analysis, we can make direct probability statements about the campaign's effectiveness and expected business impact.


In [ ]:
# Summary and save results
summary = {
    'ad_group_size': n_ad,
    'psa_group_size': n_psa,
    'ad_conversions': ad_conversions,
    'psa_conversions': psa_conversions,
    'ad_conversion_rate': cr_ad,
    'psa_conversion_rate': cr_psa,
    'posterior_mean_ad': post_mean_ad,
    'posterior_mean_psa': post_mean_psa,
    'expected_lift': (post_mean_ad - post_mean_psa) / post_mean_psa,
    'prob_ad_better': prob_ad_better,
    'prob_meaningful_lift': prob_meaningful_lift,
    'credible_interval_lower': diff_ci_lower,
    'credible_interval_upper': diff_ci_upper,
    'expected_incremental_conversions': expected_incremental_conversions,
    'expected_incremental_revenue': expected_incremental_revenue,
    'revenue_ci_lower': revenue_ci_lower,
    'revenue_ci_upper': revenue_ci_upper
}

import json
with open('bayesian_results.json', 'w') as f:
    json.dump({k: float(v) if isinstance(v, (np.integer, np.floating)) else v 
              for k, v in summary.items()}, f, indent=2)

print("\n" + "="*80)
print("BAYESIAN ANALYSIS SUMMARY")
print("="*80)
print(f"\nProbability Ad > PSA: {prob_ad_better:.2%}")
print(f"Expected Lift: {(post_mean_ad - post_mean_psa) / post_mean_psa * 100:.4f}%")
print(f"Expected Incremental Revenue: ${expected_incremental_revenue:,.2f}")
print(f"95% Credible Interval: [{diff_ci_lower:.6f}, {diff_ci_upper:.6f}]")
print(f"\n{interpretation} that Ad group is superior")
print("\n✅ Results saved to 'bayesian_results.json'")
print("="*80)


In [ ]:
# Assume a value per conversion (e.g., $100)
value_per_conversion = 100

# Expected incremental conversions
expected_incremental_conversions = (post_mean_ad - post_mean_psa) * n_ad

# Expected incremental revenue
expected_incremental_revenue = expected_incremental_conversions * value_per_conversion

print("EXPECTED BUSINESS IMPACT")
print("="*60)
print(f"Value per conversion: ${value_per_conversion}")
print(f"Expected incremental conversion rate: {post_mean_ad - post_mean_psa:.6f}")
print(f"Expected incremental conversions: {expected_incremental_conversions:.2f}")
print(f"Expected incremental revenue: ${expected_incremental_revenue:,.2f}")

# Distribution of expected revenue
revenue_samples = (posterior_ad_samples - posterior_psa_samples) * n_ad * value_per_conversion
revenue_ci_lower = np.percentile(revenue_samples, 2.5)
revenue_ci_upper = np.percentile(revenue_samples, 97.5)

print(f"\nRevenue Distribution:")
print(f"  Mean: ${revenue_samples.mean():,.2f}")
print(f"  95% Credible Interval: [${revenue_ci_lower:,.2f}, ${revenue_ci_upper:,.2f}]")
